In [44]:
import joblib
import pandas as pd
import numpy as np

# PREDICCION ABRIL

In [54]:
# nuevos datos
abril = pd.read_csv(r'C:\Users\REPO\Tec2025\HackForHer\datos_marzo.csv')

In [56]:
# Cargar modelo y scaler
model = joblib.load('modelo_sinimputar.pkl')

# Preprocesamiento (ejemplo)
# Guardar el ID en una variable separada
ids = abril[['cooler_id']].copy()

nuevos = abril[['door_opens', 'open_time', 'compressor', 'power',
       'on_time', 'min_voltage', 'max_voltage', 'temperature',
       'warning', 'year', 'month']]  # Asegúrate de que solo estén los features que entrenaste

# Predecir
pred = model.predict(nuevos)
proba = model.predict_proba(nuevos)[:, 1]

# Unir resultados con el ID
resultado = abril.copy()
resultado['pred'] = pred
resultado['proba'] = proba

In [53]:
resultado.to_csv("predicciones_abril.csv")

# Prediccion MAYO

In [57]:
mayo = resultado.copy()
ids = mayo[['cooler_id']].copy()

nuevos_mayo = mayo[['door_opens', 'open_time', 'compressor', 'power',
       'on_time', 'min_voltage', 'max_voltage', 'temperature',
       'warning', 'year', 'month']]  # Asegúrate de que solo estén los features que entrenaste

# Predecir
pred_mayo = model.predict(nuevos_mayo)
proba_mayo = model.predict_proba(nuevos_mayo)[:, 1]

# Unir resultados con el ID
resultado_mayo = mayo.copy()
resultado_mayo['pred'] = pred_mayo
resultado_mayo['proba'] = proba

In [58]:
# Ahora tienes predicciones con el cooler_id preservado
print(resultado_mayo[['cooler_id', 'pred', 'proba']].head())

                                           cooler_id  pred     proba
0  767436ca4a132715e8fe6cff5ad1f2facbf0584aafd715...     0  0.181667
1  767436ca4a132715e8fe6cff5ad1f2facbf0584aafd715...     0  0.130000
2  1a7da0241d9242b0ae9f019085d7114ff80976dbea5954...     0  0.161786
3  1a7da0241d9242b0ae9f019085d7114ff80976dbea5954...     0  0.231786
4  1a7da0241d9242b0ae9f019085d7114ff80976dbea5954...     0  0.276786


teniendo la probabilidad diaria, puedo usar probabilidad complementaria para encontrar la proba mensual

In [61]:
# Calcular probabilidad mensual por cooler_id
proba_mensual = resultado_mayo.groupby('cooler_id')['proba'].apply(lambda p: 1 - np.prod(1 - p))

# Convertir a DataFrame si deseas exportarlo o analizarlo
proba_mensual_df = proba_mensual.reset_index(name='proba_mensual')

# Mostrar resultado
print(proba_mensual_df.head())

proba_mensual_df.to_csv('probabilidad.csv')


                                           cooler_id  proba_mensual
0  0016ec5ff0bab59a840bfb35f19432270b06e821b6807f...       0.984959
1  00190192bd70e98a2399c87c883f9abd122ddf1ea91515...       0.624725
2  001eacd5c959eff51b42e8855473e89be89b46e8a5d0d8...       0.797904
3  0025f7d0ed62936e9107af28a61d68a8a34606d1690dab...       0.912829
4  0026be8ea32cc11fe427ed626def3096f35d0fbdb4da1c...       0.683571
